In [4]:
import sys
if "/Users/raymond/code/FinalProject563" not in sys.path:
    sys.path.append("/Users/raymond/code/FinalProject563")
import pandas
import numpy as np
import torch
import matplotlib.pyplot as plt
import random

import connect_four
import mcts
import connect_net
import connect_plot
import evaluation
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
nnet = connect_net.ConnectNet()
strategy = evaluation.NetworkStrategy(nnet=nnet, c=1, num_mcts_sims=1024, random_moves=4)
game_cache = connect_four.GameCache()
strategy.setup(game_cache=game_cache)

state = connect_four.start_state()
rep = connect_four.to_rep(state=state)

In [14]:
import cProfile
cProfile.run('strategy.simulate(state=state, rep=rep)', sort="tottime")

         219363 function calls (199779 primitive calls) in 1.707 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
6208/1024    0.397    0.000    1.698    0.002 mcts.py:89(search)
5184/1024    0.378    0.000    1.582    0.002 mcts.py:32(search_action)
     2048    0.274    0.000    0.274    0.000 {built-in method conv2d}
     6557    0.058    0.000    0.058    0.000 {built-in method rsub}
     1024    0.051    0.000    0.075    0.000 connect_four.py:32(game_ended)
     4096    0.044    0.000    0.044    0.000 {built-in method tanh}
     6208    0.041    0.000    0.041    0.000 {method 'sum' of 'torch._C._TensorBase' objects}
     2048    0.039    0.000    0.039    0.000 {built-in method torch._C._nn.linear}
     1373    0.038    0.000    0.064    0.000 connect_four.py:45(next_state)
     1024    0.031    0.000    0.491    0.000 connect_net.py:35(forward)
     5184    0.029    0.000    0.029    0.000 {built-in method argmax}
 

In [38]:
examples = mcts.policy_iteration(nnet=nnet, num_iterations=1, num_episodes=1, num_mcts_sims=128)


Iteration: 0::0,

In [4]:
test_state = np.zeros([6,7], dtype=np.int32)
test_state[5,3] = 1
test_state[5,2] = -1
test_state[4,3] = 1
test_state[3,3] = 1
test_state[2,3] = 1
test_state[0,0] = 1
test_state[1,0] = -1
test_state[2,0] = 1
test_state[3,0] = -1
test_state[4,0] = 1
test_state[5,0] = -1
test_state_tensor = torch.Tensor(test_state).reshape(6, 7)

test = {}
test["12948731587817346817264812764817623"] = 10

In [11]:
N = torch.Tensor([1, 2, 3, 4, 5, 6, 7])
Q = torch.Tensor([1, 2, 3, 4, 5, 6, 7])
P = torch.Tensor([1, 2, 3, 4, 5, 6, 7])
c = 1

In [24]:
def old():
    max_u, best_a = -torch.inf, -1
    # for a in self.nc.actions(state=state, rep=rep):
    for a in connect_four.get_valid_actions(state=test_state_tensor):
        visits = 1 + N[a]
        u_explore = (N.sum() + 1).sqrt() / visits
        u = Q[a] / visits + c * P[a] * u_explore
        if u > max_u:
            max_u = u
            best_a = a
        print(a)
    a = best_a
    return a
old()

1
2
3
4
5
6


6

In [30]:
1-test_state_tensor[0,:]

visits = 1 + N
u_explore = (N.sum() + 1).sqrt() / visits
u = 1 + Q / visits + c * P * u_explore
u = u * (1 - test_state_tensor[0, :])
int(torch.argmax(u))

6

In [31]:
test_state_tensor[0, :]

tensor([1., 0., 0., 0., 0., 0., 0.])

In [27]:
u*(1-test_state_tensor[0,:])

tensor([0.0000, 5.2568, 5.7889, 6.1081, 6.3210, 6.4730, 6.5870])